In [1]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder, DepositTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests

import json
import pickle

import csv
import os
import time
from pprint import pprint


In [2]:
connections = {
    "mainnet": {"iconservice": "https://ctz.solidwallet.io", "nid": 1},
    "berlin": {"iconservice": "https://berlin.net.solidwallet.io", "nid": 7},
    "sejong": {"iconservice": "https://sejong.net.solidwallet.io", "nid": 83}}


In [3]:
icon_service_berlin= IconService(HTTPProvider(connections["berlin"]["iconservice"],3))
icon_service_sejong=IconService(HTTPProvider(connections["sejong"]["iconservice"],3))
NID_b = connections["berlin"]["nid"]
NID_s = connections["sejong"]["nid"]

In [4]:


@retry(JSONRPCException, tries=10, delay=1, back_off=1)
def get_tx_result(_tx_hash,icon_service):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result


In [5]:
private="eef769ccffeda60de514dc5e41c3d1c354efcc6ae58d8406ee03ad06b188e5b3"
wallet = KeyWallet.load(bytes.fromhex(private))
wallet.get_address()

'hxad4edfdb6d5834a4350a74bea5448535385c07d5'

In [6]:
contracts = {
    "oracleSejong":"cx900e2d17c38903a340a0181523fa2f720af9a798",
    "oracleBerlinProxy":"cx36a55b74aca43a9db9d5a8fc876c76d04daa85a2",
    "oracleBerlinRef":"cx4a90a6c841dc056c10edc8080bcb60c415bcee0c"
}

In [14]:
# cell 4
# initialized all methods
def get_status_and_print(to, method, response) -> int:
    """
    get status and print response
    :param to:
    :param method:
    :param response:
    :return: status
    """
    status = response['status']
    print(f'SCORE {to} ::: Method {method} ::: Status: {status}')
    if len(response["eventLogs"]) > 0:
        for item in response["eventLogs"]:
            print(f'{item} \n')
    if status == 0:
        print(f'Failure: {response["failure"]}')
    return status


def send_tx(_to, _method, _params, _value) -> int:
    """
    send transaction to _to
    :param _to: destination conract name
    :param _method: method
    :param _params:  parameters
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    NID = NID_b if "Berlin" in _to else NID_s
    icon_service = icon_service_berlin if "Berlin" in _to else icon_service_sejong
    print(f'Calling {_method}, with parameters {_params} on the {_to} contract.')
    transaction = CallTransactionBuilder() \
        .from_(wallet.get_address()) \
        .to(contracts[_to]) \
        .value(_value) \
        .step_limit(30000000000) \
        .nid(NID) \
        .nonce(100) \
        .method(_method) \
        .params(_params) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    response = get_tx_result(tx_hash,icon_service)
    status = get_status_and_print(_to, _method, response)
    return status,response
    
def readOnlyCall(_to, _method, _params):
    """
    send transaction to _to
    :param _to: destination conract name
    :param _method: method
    :param _params:  parameters
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    icon_service = icon_service_berlin if "Berlin" in _to else icon_service_sejong
    _call = CallBuilder() \
        .from_(wallet.get_address()) \
        .to(contracts[_to]) \
        .method(_method) \
        .params(_params) \
        .build()
    result = icon_service.call(_call)
    return result

In [8]:
pprint(contracts)


{'oracleBerlinProxy': 'cx36a55b74aca43a9db9d5a8fc876c76d04daa85a2',
 'oracleBerlinRef': 'cx4a90a6c841dc056c10edc8080bcb60c415bcee0c',
 'oracleSejong': 'cx900e2d17c38903a340a0181523fa2f720af9a798'}


In [15]:
def getSejongPrice(base):
    price = readOnlyCall("oracleSejong","get_reference_data",{"_base":base,"_quote":"USD"})
    return price
def getBerlinPrice(base):
    price = readOnlyCall("oracleBerlinProxy","get_reference_data",{"_base":base,"_quote":"USD"})
    return price


In [48]:
print(0x5de05d340df00)
print(1600419716)

1651491484000000
1600419716


In [52]:
usdcPriceSej = getSejongPrice("USDC")
usdcPriceBer = getSejongPrice("USDC")
print(usdcPriceSej)
print(usdcPriceBer)

{'last_update_base': '0x5de05e55048c0', 'last_update_quote': '0x5de05ec89690f', 'rate': '0xddffe57f379e000'}
{'last_update_base': '0x5de05e55048c0', 'last_update_quote': '0x5de05eca7ea1a', 'rate': '0xddffe57f379e000'}


In [51]:
priceBerlin = getBerlinPrice("ICX")
priceSejong = getSejongPrice("ICX")
print(priceBerlin)
print(priceSejong)

{'last_update_base': '0x5de05e55048c0', 'last_update_quote': '0x5de05ebb8799d', 'rate': '0x89721386bf78000'}
{'last_update_base': '0x5de05e55048c0', 'last_update_quote': '0x5de05ebc2addb', 'rate': '0x89721386bf78000'}


In [63]:
# _rates =json.dumps([626897348])
# _request_ids =json.dumps([7840506])
# _resolve_times = json.dumps([1651487885])
# _symbols=json.dumps(["ICX"])
# params ={"_rates":_rates,"_request_ids":_request_ids,"_resolve_times":_resolve_times,"_symbols":_symbols}
# relay = send_tx("oracleBerlinRef","relay",params,0)

Calling relay, with parameters {'_rates': '[626897348]', '_request_ids': '[7840506]', '_resolve_times': '[1651487885]', '_symbols': '["ICX"]'} on the oracleBerlinRef contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing: Executing'}


{'code': -31003, 'message': 'Executing: Executing'} (Json rpc error), Retrying in 1 seconds...
SCORE oracleBerlinRef ::: Method relay ::: Status: 1
{'scoreAddress': 'cx4a90a6c841dc056c10edc8080bcb60c415bcee0c', 'indexed': ['RefDataUpdate(str,int,int,int)'], 'data': ['ICX', '0x255db1c4', '0x626fb48d', '0x77a2fa']} 



In [49]:
def updatePrice():
    sejongPrice =getSejongPrice("ICX")
    usdcPrice =getSejongPrice("USDC")
    lastUpdate = int(sejongPrice["last_update_base"],0)//10**6
    print(sejongPrice)
    _rates =json.dumps([int(sejongPrice["rate"],0)// 10**9,int(usdcPrice["rate"],0)// 10**9])
    _request_ids =json.dumps([78405089,78405089])
    _resolve_times = json.dumps([lastUpdate,lastUpdate])
    _symbols=json.dumps(["ICX","USDC"])
    params ={"_rates":_rates,"_request_ids":_request_ids,"_resolve_times":_resolve_times,"_symbols":_symbols}
    relay = send_tx("oracleBerlinRef","relay",params,0)
    return relay


In [50]:
relay=updatePrice()

{'last_update_base': '0x5de05e55048c0', 'last_update_quote': '0x5de05eb1a9275', 'rate': '0x89721386bf78000'}
Calling relay, with parameters {'_rates': '[619000000, 999797296]', '_request_ids': '[78405089, 78405089]', '_resolve_times': '[1651491787, 1651491787]', '_symbols': '["ICX", "USDC"]'} on the oracleBerlinRef contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing: Executing'}


{'code': -31003, 'message': 'Executing: Executing'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing: Executing'}


{'code': -31003, 'message': 'Executing: Executing'} (Json rpc error), Retrying in 1 seconds...
SCORE oracleBerlinRef ::: Method relay ::: Status: 1
{'scoreAddress': 'cx4a90a6c841dc056c10edc8080bcb60c415bcee0c', 'indexed': ['RefDataUpdate(str,int,int,int)'], 'data': ['ICX', '0x24e530c0', '0x626fc3cb', '0x4ac5de1']} 

{'scoreAddress': 'cx4a90a6c841dc056c10edc8080bcb60c415bcee0c', 'indexed': ['RefDataUpdate(str,int,int,int)'], 'data': ['USDC', '0x3b97b230', '0x626fc3cb', '0x4ac5de1']} 



In [41]:
a=0
price = getBerlinPrice("ICX")
while a==0:
    newPrice = getSejongPrice("ICX")
    print("No Price Change")
    if newPrice["rate"] != price["rate"]:
        updatePrice()
        price=newPrice
        
    


No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price

Raised JSONRPCException while returning the custom response. Error message: {'code': -31002, 'message': 'Pending: Pending'}


{'code': -31002, 'message': 'Pending: Pending'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31002, 'message': 'Pending: Pending'}


{'code': -31002, 'message': 'Pending: Pending'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing: Executing'}


{'code': -31003, 'message': 'Executing: Executing'} (Json rpc error), Retrying in 1 seconds...
SCORE oracleBerlinRef ::: Method relay ::: Status: 1
{'scoreAddress': 'cx4a90a6c841dc056c10edc8080bcb60c415bcee0c', 'indexed': ['RefDataUpdate(str,int,int,int)'], 'data': ['ICX', '0x25317c00', '0x626fc17f', '0x4ac5de1']} 

{'scoreAddress': 'cx4a90a6c841dc056c10edc8080bcb60c415bcee0c', 'indexed': ['RefDataUpdate(str,int,int,int)'], 'data': ['USDC', '0x3b9aca00', '0x626fc17f', '0x4ac5de1']} 

No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change
No Price Change

KeyboardInterrupt: 